In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, Concatenate, Input
from keras.callbacks import EarlyStopping
import word2vec_utils as w2v
import data
import numpy as np
import re
from data_utils import split_dataset 
from chat_constants import *


Using Theano backend.


In [2]:
model = w2v.initialize()

In [3]:
A1, B, A2 = data.load_data()


In [4]:
A1_train,B_train,A2_train = w2v.get_n_training_data(20000,A1,B,A2)
A1B_train = np.concatenate((A1_train,B_train),axis=1)

In [5]:
def null_punisher(y_true, y_pred ):
    '''Provides stronger incentive to avoid Null'''
    L = 3
    error = keras.losses.cosine_proximity( y_true,y_pred)
    if y_true != BLANK:
        error *= L
    return error

In [6]:
LSTM_DROPOUT = 0.1

In [ ]:
A1_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

A1_layer1 = LSTM(EMBED_DIM,return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A1 = A1_layer1(A1_input)

In [ ]:
B_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

B_layer1 = LSTM(EMBED_DIM,return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
B = B_layer1(B_input)

In [ ]:
combined = Concatenate([A1,B])


In [ ]:
AB_Input = Input(shape =(MAX_SENT_LENGTH*2,EMBED_DIM))

AB_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer1", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer2 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer2", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer3 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer3", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

AB_output = AB_layer3(AB_layer2(AB_layer1(AB_Input)))

chat_model = Model(inputs = [ AB_Input], outputs = [AB_output])
chat_model.compile(loss=null_punisher,
              optimizer='Adam',
              metrics=['accuracy']
              )
early_stopping = EarlyStopping( monitor = 'val_acc',patience = 2)

In [ ]:
chat_model.load_weights('chat_net.h5',by_name=True)
chat_model.fit([A1B_train], A2_train,
          batch_size=200, epochs=20,
          validation_split = 0.025 )
chat_model.save('chat_net.h5')

Train on 19500 samples, validate on 500 samples
Epoch 1/20
19500/19500 [==============================] - 785s - loss: -0.0098 - acc: 0.8399 - val_loss: -0.0099 - val_acc: 0.8833
Epoch 2/20
19500/19500 [==============================] - 1081s - loss: -0.0099 - acc: 0.8843 - val_loss: -0.0099 - val_acc: 0.9500
Epoch 3/20
19500/19500 [==============================] - 1644s - loss: -0.0099 - acc: 0.9070 - val_loss: -0.0099 - val_acc: 0.9667
Epoch 4/20
19500/19500 [==============================] - 1811s - loss: -0.0099 - acc: 0.9171 - val_loss: -0.0099 - val_acc: 0.9667
Epoch 5/20
19500/19500 [==============================] - 2031s - loss: -0.0099 - acc: 0.9234 - val_loss: -0.0099 - val_acc: 0.9833
Epoch 6/20
19500/19500 [==============================] - 1982s - loss: -0.0099 - acc: 0.9287 - val_loss: -0.0099 - val_acc: 0.9667
Epoch 7/20
19500/19500 [==============================] - 1972s - loss: -0.0099 - acc: 0.9333 - val_loss: -0.0099 - val_acc: 1.0000
Epoch 8/20
19500/19500 [=====

In [12]:
a = {}
a['__BLANK__'] = BLANK
a['__BLANK__']

import pickle
with open('unknown_words_stored.pkl', 'w+') as f:
    pickle.dump(a, f)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1]

In [15]:
EMBED_DIM = len(BLANK)

In [11]:
len(A1_train[1])

30

In [41]:
len(A1)

106455

In [4]:
A2.shape

NameError: name 'A2' is not defined